In [1]:
# Importando as bibliotecas necessárias

import requests
import json
import pandas as pd
from dotenv import load_dotenv
import os

In [2]:
# Carregando as credenciais de acesso
load_dotenv()

client_id = os.environ["email"]
client_secret = os.environ["senha"]

In [19]:
# Coletando o token de acesso na Parcela Mais
# É necessário fazer uma chamada POST com as credenciais de acesso para poder receber o access token

url = "https://parcelamais.com/api/1.1/wf/auth"


headers = {
    "accept": "application/json",
    "content-type": "application/json"
}


payload = {
    "client_id": client_id,
    "client_secret": client_secret
}

response = requests.post(
    url,
    headers = headers,
    json = payload
    )

print(response.text)

{
    "status": "success",
    "response": {
        "token": "1672927784034x656559345802010800",
        "user_id": "1672840071855x815545991419960300",
        "expires": 86400
    }
}


In [20]:
# Parseando o json para obter o valor do token
# O token tem expiração de 24h, portanto precisa ser renovado sempre

json_parsed = json.loads(response.text)

token = json_parsed["response"]["token"]

In [22]:
# Puxando os dados da tabela Proposal no banco de dados do Bubble usando o token de chamada

tabela_final = pd.DataFrame()
cursor = 0

# Fazendo várias requisições somando sempre +100 no cursor até que o resultado da chamada seja menor do que 1

while True:
    
    url = f'https://parcelamais.com/api/1.1/obj/proposal?cursor={cursor}'
    
    headers_get = {
        "accept": "application/json",
        "authorization": "Bearer " + token
    }
    
    response = requests.get(
        url,
        headers = headers_get
    )
    
    informacoes = response.json()
    
    tabela = pd.DataFrame(informacoes['response']['results'])
    
    if len(informacoes['response']['results']) < 1:
        break
    
    tabela_final = pd.concat([tabela_final, tabela])
    
    cursor = cursor + 100
    
#display(tabela_final)
tabela_final.to_csv('[parcelamais]get_data_bubble.csv', index = False)